In [50]:
import numpy as np
import torch, argparse, json
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import torch.autograd as autograd
import random

learning_rate = 0.01
epochs = 100
display_epochs = 10
test_percent = 10 #percentage out of 100
batch_size = 50

In [51]:
#Converts raw data into usable integer array
data_raw = open("data\in.txt","r")
data = []

for line in data_raw:
    numString = list(line)
    #print(line[0])
    data.append(numString[:-1])

for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j]=='+':
            data[i][j]=1
        else:
            data[i][j]=-1

size = len(data[0])
test_L = int(len(data)*test_percent/100)
train = data[:-test_L]
test = data[test_L:]

js = np.random.rand(len(data),size)
for i in range(len(js)):
    for j in range(len(js[i])):
        if js[i][j]>=0.5:
            js[i][j]=1
        else:
            js[i][j]=-1

lamb = np.random.rand(size)
for i in range(size):
    if lamb[i]>=0.5:
        lamb[i]=1
    else:
        lamb[i]=-1

[ 1. -1.  1.  1.]


In [52]:
Z = 2**size

js = []
s_i = np.random.rand(size)
for i in range(size):
    if s_i[i]>=0.5:
        s_i[i]=1
    else:
        s_i[i]=-1
js.append(s_i)

In [53]:
def MC(s_i): #Markov chain Monte Carlo
    #randomly generate state
    s_j = np.random.rand(size)
    for i in range(size):
        if s_j[i]>=0.5:
            s_j[i]=1
        else:
            s_j[i]=-1
    #calculate energy of initial state
    E_i = Energy(s_i)
    #calculate energy of randomly generated state
    E_j = Energy(s_j)
    #check for usable value
    if E_j<E_i:
        js.append(s_j)
        s_i = s_j
    else:
        threshold = exp(E_i-E_j)
        rando = random.uniform(0,1)
        if rando<threshold:
            js.append(s_j)
            s_i = s_j
        else:
            MC(s_i)
        
def Energy(s):#calculate the energy of a state
    E = 0
    for i in range(size):
        if i!=(size-1):
            E+=lamb*s[i]*s[i+1]
        else:
            E+=lamb*s[i]*s[0]
    return -E

In [54]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv1= nn.Conv1d(size,size,size)
        self.fc1= nn.Linear(size, size)
        
    def forward(self, x):
        y = func.relu(self.conv1(x))
        y = func.max_pool1d(self.lin(h))
        return y
    
    def reset(self):
        self.conv1.reset_parameters()
        self.fc1.reset_parameters()

In [55]:
model = Net().to(torch.device("cpu"))
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss = nn.NLLLoss(reduction='mean')

In [57]:
obj_vals= []
obj_accs= []
cross_vals= []
cross_accs= []

model.reset()

for epoch in range(epochs):
    #Training
    batch_index = 0
    correct = 0
    temp_vals= []
    for batch_index in batch(train, batch_size):
        x_t = batch_index[:,:-1]
        y_t = batch_index[:,-1]
        x_t = np.array(x_t, dtype = np.float32)
        inputs = torch.from_numpy(x_t)
        targets = torch.from_numpy(y_t)
        targets = targets.type(torch.LongTensor)
        
        temp_val = 10**(loss(model.forward(inputs), targets))

        optimizer.zero_grad()
        temp_val.backward()
        optimizer.step()

        temp_vals.append(temp_val.item())
        output = torch.max(model(inputs), dim=1)[1].squeeze()
        correct += (output == targets).float().sum()
        
    obj_val = np.average(temp_vals)
    obj_vals.append(obj_val.item())
    obj_accs.append(100*correct/len(train))

    if (epoch+1) % display_epochs == 0:
        print ('Epoch [{}/{}]\t Training Loss: {:.4f}'.format(epoch+1, epochs, obj_val.item()))
        print ('Epoch [{}/{}]\t Training Accuracy: {:.4f}%'.format(epoch+1, epochs, obj_accs[-1]))

    #Checking loss at each step to compare
    with torch.no_grad(): 
        correct_c = 0
        # don't track calculations in the following scope for the purposes of gradients
        inputs = torch.from_numpy(x_test)
        targets = torch.from_numpy(y_test)
        targets = targets.type(torch.LongTensor)
        cross_val = 10**(loss(model.forward(inputs), targets))
        output_c = torch.max(model(inputs), dim=1)[1].squeeze()
        correct_c += (output_c == targets).float().sum()
        cross_vals.append(cross_val)
        cross_accs.append(100*correct_c/len(test))

    if (epoch+1) % display_epochs == 0:
        print ('Epoch [{}/{}]\t Test Loss: {:.4f}'.format(epoch+1, epochs, cross_val.item()))
        print ('Epoch [{}/{}]\t Test Accuracy: {:.4f}% \n'.format(epoch+1, epochs, cross_accs[-1]))

print('\n~~~~~~~~~~~~~~~~~~PERFORMANCE REPORT~~~~~~~~~~~~~~~~~~')
print('Final training performance: \n Loss: {:.4f} \t Accuracy: {:0.2f}%'.format(obj_vals[-1],obj_accs[-1]))
print('Final testing performance: \n Loss: {:.4f} \t Accuracy: {:0.2f}%'.format(cross_vals[-1],cross_accs[-1]))

TypeError: 'int' object is not callable